# This colab notebook must be run on a **P100** GPU instance otherwise it will crash. Use the Cell-1 to ensure that it has a **P100** GPU instance 

Cell-1: Ensure the required gpu instance (P100)

In [ ]:
#no.of sockets i.e available slots for physical processors
!lscpu | grep 'Socket(s):'
#no.of cores each processor is having 
!lscpu | grep 'Core(s) per socket:'
#no.of threads each core is having
!lscpu | grep 'Thread(s) per core'
#GPU count and name
!nvidia-smi -L
#use this command to see GPU activity while doing Deep Learning tasks, for this command 'nvidia-smi' and for above one to work, go to 'Runtime > change runtime type > Hardware Accelerator > GPU'
!nvidia-smi

Cell-2: Add Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Cell-3: Install Required Dependencies

In [ ]:
!pip install efficientnet_pytorch==0.7.0 
!pip install albumentations==0.4.5
!pip install torch==1.6.0+cu101 torchvision==0.7.0+cu101 -f https://download.pytorch.org/whl/torch\_stable.html -q\

Cell-4: Run this cell to generate current fold weight ( Estimated Time for training this fold is around 6 hours 50 minutes )

In [ ]:
import sys
sys.path.insert(0, "/content/gdrive/My Drive/zindi_cgiar_wheat_growth_stage_challenge/src_wd")

from dataset import *
from model import *
from trainer import *
from utils import *

import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from torch.utils.data import DataLoader

config = {
    'n_folds': 5,
    'random_seed': 4690,
    'run_fold': 0,
    'batch_size': 86,
    'n_core': 0,
    'model_name': 'efficientnet-b2',
    'global_dim': 1408,
    'weight_saving_path': '/content/gdrive/My Drive/zindi_cgiar_wheat_growth_stage_challenge/train_wd_effnet_b2/weights/',
    'resume_checkpoint_path': None,
    'lr': 0.01,
    'total_epochs': 100,
    }


if __name__ == '__main__':
    set_random_state(config['random_seed']) 
    
    imgs = np.load('/content/gdrive/My Drive/zindi_cgiar_wheat_growth_stage_challenge/zindi_npy_data/train_imgs.npy')
    labels = np.load('/content/gdrive/My Drive/zindi_cgiar_wheat_growth_stage_challenge/zindi_npy_data/train_labels.npy')
    labels = labels - 1        
    skf = StratifiedKFold(n_splits=config['n_folds'], shuffle=True, random_state=config['random_seed'])    
    for fold_number, (train_index, val_index) in enumerate(skf.split(X=imgs, y=labels)):
        if fold_number != 0:
            continue
                        
        train_dataset = ZCDataset(
                            imgs[train_index],
                            labels[train_index],
                            test=False
                            )               
        train_loader = DataLoader(
                            train_dataset, 
                            batch_size=config['batch_size'], 
                            shuffle=True, 
                            num_workers=config['n_core'],
                            drop_last=True,
                            pin_memory=True,
                            )        
        val_dataset = ZCDataset(
                            imgs[val_index],
                            labels[val_index],                          
                            test=True,
                            )        
        val_loader = DataLoader(
                            val_dataset, 
                            batch_size=config['batch_size'], 
                            shuffle=False, 
                            num_workers=config['n_core'],
                            pin_memory=True,
                            )
        
        del imgs,labels

        model = CNN_Model(config['model_name'], config['global_dim'])        
        
        args = { 
                'model': model,
                'Loaders': [train_loader,val_loader],
                'metrics': {'Loss':AverageMeter, 'f1_score':PrintMeter, 'rmse':PrintMeter},                       
                'checkpoint_saving_path': config['weight_saving_path'],
                'resume_train_from_checkpoint': False,
                'resume_checkpoint_path': config['resume_checkpoint_path'],
                'lr': config['lr'],
                'fold': fold_number,
                'epochsTorun': config['total_epochs'],
                'batch_size': config['batch_size'],
                'test_run_for_error': False,
                'problem_name': 'zindi_cigar',
                }         
        Trainer = ModelTrainer(**args)
        Trainer.fit()        